In [2]:
# Running example from the following page
# https://realpython.com/linear-programming-python/
from scipy.optimize import linprog

In [3]:
obj = [-1, -2]
lhs_ineq = [[ 2,  1],[-4,  5],[ 1, -2]]
rhs_ineq = [20, 10, 2]
lhs_eq = [[-1, 5]]
rhs_eq = [15]

In [4]:
bnd = [(0, float("inf")),(0, float("inf"))]

In [5]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
               A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
               method="highs")

In [6]:
opt

     con: array([0.])
     fun: -16.818181818181817
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([ 0.        , 18.18181818,  3.36363636])
  status: 0
 success: True
       x: array([7.72727273, 4.54545455])

In [7]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [9]:
model = LpProblem(name="small-problem", sense=LpMaximize)

In [10]:
# Initialize the decision variables
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

In [11]:
# not sure about this 
expression = 2 * x + 4 * y
constraint = 2 * x + 4 * y >= 8

In [12]:
# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

In [13]:
# Add the objective function to the model
obj_func = x + 2 * y
model += obj_func

In [14]:
# Add the objective function to the model - shorter notation
# model += x + 2 * y

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [15]:
model

small-problem:
MAXIMIZE
1*x + 2*y + 0
SUBJECT TO
red_constraint: 2 x + y <= 20

blue_constraint: 4 x - 5 y >= -10

yellow_constraint: - x + 2 y >= -2

green_constraint: - x + 5 y = 15

VARIABLES
x Continuous
y Continuous

In [16]:
# Solve the problem
status = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/saturncloud/envs/saturn/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/05c36dd5b0f34008a49f0469a4edd244-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/05c36dd5b0f34008a49f0469a4edd244-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 20 RHS
At line 25 BOUNDS
At line 26 ENDATA
Problem MODEL has 4 rows, 2 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-4) rows, 0 (-2) columns and 0 (-8) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 16.818182
After Postsolve, objective 16.818182, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 16.81818182 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock sec

In [17]:
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [18]:
print(f"objective: {model.objective.value()}")

objective: 16.8181817


In [19]:
for var in model.variables():
     print(f"{var.name}: {var.value()}")

x: 7.7272727
y: 4.5454545


In [20]:
for name, constraint in model.constraints.items():
     print(f"{name}: {constraint.value()}")

red_constraint: -9.99999993922529e-08
blue_constraint: 18.181818300000003
yellow_constraint: 3.3636362999999996
green_constraint: -2.0000000233721948e-07


In [1]:
# Now using examples from:
# https://mlabonne.github.io/blog/posts/2022-03-05-linear_programming.html
from ortools.linear_solver import pywraplp

In [2]:
# Create the linear solver using the CBC backend
solver = pywraplp.Solver('Maximize army power', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# 1. Create the variables we want to optimize
swordsmen = solver.IntVar(0, solver.infinity(), 'swordsmen')
bowmen = solver.IntVar(0, solver.infinity(), 'bowmen')
horsemen = solver.IntVar(0, solver.infinity(), 'horsemen')

# 2. Add constraints for each resource
solver.Add(swordsmen*60 + bowmen*80 + horsemen*140 <= 1200)
solver.Add(swordsmen*20 + bowmen*10 <= 800)
solver.Add(bowmen*40 + horsemen*100 <= 600)

# 3. Maximize the objective function
solver.Maximize(swordsmen*70 + bowmen*95 + horsemen*230)

# Solve problem
status = solver.Solve()

# If an optimal solution has been found, print results
if status == pywraplp.Solver.OPTIMAL:
  print('================= Solution =================')
  print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
  print()
  print(f'Optimal value = {solver.Objective().Value()} 💪power')
  print('Army:')
  print(f' - 🗡️Swordsmen = {swordsmen.solution_value()}')
  print(f' - 🏹Bowmen = {bowmen.solution_value()}')
  print(f' - 🐎Horsemen = {horsemen.solution_value()}')
else:
  print('The solver could not find an optimal solution.')

================= Solution =================
Solved in 51.00 milliseconds in 0 iterations

Optimal value = 1800.0 💪power
Army:
 - 🗡️Swordsmen = 6.0
 - 🏹Bowmen = 0.0
 - 🐎Horsemen = 6.0
